In [1]:
import pandas as pd
import numpy as np
import csv 
import os

In [2]:
PATH = "./disgenet/"
filename = "all_gene_disease_associations.tsv"

In [3]:
df = pd.read_csv(PATH+filename, sep = '\t')
df.head()

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.857,0.172,C0001418,Adenocarcinoma,group,C04,Neoplastic Process,0.01,NaN,2008.0,2008.0,1,0,LHGDN
1,1,A1BG,0.857,0.172,C0002736,Amyotrophic Lateral Sclerosis,disease,C10;C18,Disease or Syndrome,0.01,1.0,2009.0,2009.0,1,0,BEFREE
2,1,A1BG,0.857,0.172,C0013080,Down Syndrome,disease,C10;C16,Disease or Syndrome,0.01,1.0,2011.0,2011.0,1,0,BEFREE
3,1,A1BG,0.857,0.172,C0017636,Glioblastoma,disease,C04,Neoplastic Process,0.01,1.0,2014.0,2014.0,1,0,BEFREE
4,1,A1BG,0.857,0.172,C0019209,Hepatomegaly,phenotype,C06;C23,Finding,0.30,NaN,2017.0,2017.0,1,0,CTD_human


In [4]:
print(len(df), df.columns)

628685 Index(['geneId', 'geneSymbol', 'DSI', 'DPI', 'diseaseId', 'diseaseName',
       'diseaseType', 'diseaseClass', 'diseaseSemanticType', 'score', 'EI',
       'YearInitial', 'YearFinal', 'NofPmids', 'NofSnps', 'source'],
      dtype='object')


In [5]:
#find row associated to disease Malignant Mesothelioma with id C0345967

In [6]:
df.loc[df['diseaseId'] == 'C0345967']

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
343,10,NAT2,0.466,0.828,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.34,0.5,1995.0,2009.0,4,0,BEFREE;CTD_human
1871,31,ACACA,0.552,0.759,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2014.0,2014.0,1,0,BEFREE
6612,142,PARP1,0.432,0.862,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.32,1.0,2011.0,2011.0,2,0,BEFREE;CTD_human
8831,182,JAG1,0.467,0.724,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2001.0,2001.0,1,0,BEFREE
11117,207,AKT1,0.355,0.862,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2015.0,2015.0,1,0,BEFREE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617173,619553,MIR484,0.752,0.241,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.30,NaN,2015.0,2015.0,1,0,CTD_human
619069,693210,MIR625,0.681,0.172,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2012.0,2012.0,1,0,BEFREE
624677,100506965,PWAR6,1.000,0.069,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.30,NaN,2015.0,2015.0,1,0,CTD_human
626381,100862685,ERVK-19,0.573,0.724,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2015.0,2015.0,1,0,BEFREE


In [8]:
target = df.loc[df['diseaseName'] == 'Malignant mesothelioma']

In [9]:
target.to_csv(PATH+"malignant_mesothelioma_genes.tsv", sep = '\t')

In [10]:
t = pd.read_csv(PATH+"malignant_mesothelioma_genes.tsv", sep = '\t')

In [11]:
t.head()

,Unnamed: 0,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,343,10,NAT2,0.466,0.828,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.34,0.5,1995.0,2009.0,4,0,BEFREE;CTD_human
1,1871,31,ACACA,0.552,0.759,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2014.0,2014.0,1,0,BEFREE
2,6612,142,PARP1,0.432,0.862,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.32,1.0,2011.0,2011.0,2,0,BEFREE;CTD_human
3,8831,182,JAG1,0.467,0.724,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2001.0,2001.0,1,0,BEFREE
4,11117,207,AKT1,0.355,0.862,C0345967,Malignant mesothelioma,disease,C04;C08,Neoplastic Process,0.01,1.0,2015.0,2015.0,1,0,BEFREE
